# LLM Preference Prediction - Baseline Notebook

This notebook implements a baseline solution for the LLM Classification challenge.
It uses a **DeBERTa-v3** model in a Cross-Encoder configuration to predict which response is preferred.

In [ ]:
# Install necessary libraries
# We only install what is missing. We avoid upgrading torch/numpy/pandas to prevent conflicts.
# We pin pyarrow and rich because 'datasets' might try to upgrade them, breaking other kaggle packages.
!pip install -q "pyarrow<20" "rich<14" transformers datasets sentencepiece

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Configuration

In [ ]:
class Config:
    MODEL_NAME = "microsoft/deberta-v3-xsmall" # Change to 'base' or 'large' for better results
    MAX_LENGTH = 512
    BATCH_SIZE = 4
    EPOCHS = 1
    LEARNING_RATE = 2e-5
    SEED = 42
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(Config.SEED)

## Dataset Class

In [ ]:
class LLMPreferenceDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=1024, is_train=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_train = is_train
        self.texts = []
        self.labels = []
        
        for _, row in df.iterrows():
            prompt = str(row['prompt'])
            res_a = str(row['response_a'])
            res_b = str(row['response_b'])
            
            # Format: [CLS] Prompt [SEP] Response A [SEP] Response B [SEP]
            text = f"{prompt} {tokenizer.sep_token} {res_a} {tokenizer.sep_token} {res_b}"
            self.texts.append(text)
            
            if self.is_train:
                if row['winner_model_a'] == 1:
                    label = 0
                elif row['winner_model_b'] == 1:
                    label = 1
                else:
                    label = 2
                self.labels.append(label)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        
        if self.is_train:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            
        return item

## Model Class

In [ ]:
class LLMPreferenceModel(nn.Module):
    def __init__(self, model_name, num_labels=3):
        super(LLMPreferenceModel, self).__init__()
        config = AutoConfig.from_pretrained(model_name)
        config.num_labels = num_labels
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

## Training Loop

In [ ]:
def train():
    # Load Data
    # Assuming files are in the current directory
    if os.path.exists("train.csv"):
        df = pd.read_csv("train.csv")
        # df = df.sample(n=2000, random_state=42) # Uncomment for quick testing
    else:
        print("train.csv not found. Please upload dataset.")
        return

    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
    full_dataset = LLMPreferenceDataset(df, tokenizer, max_length=Config.MAX_LENGTH)
    
    train_size = int(0.9 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)
    
    model = LLMPreferenceModel(Config.MODEL_NAME)
    model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=Config.LEARNING_RATE)
    total_steps = len(train_loader) * Config.EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    best_val_loss = float('inf')
    
    for epoch in range(Config.EPOCHS):
        print(f"\nEpoch {epoch + 1}/{Config.EPOCHS}")
        model.train()
        total_train_loss = 0
        
        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            model.zero_grad()
            outputs = model(input_ids, attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()
            
        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Average Train Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        total_val_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask, labels=labels)
                total_val_loss += outputs.loss.item()
                
        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "best_model.pt")
            print("Saved Best Model")
            
    return model

## Inference

In [ ]:
def inference(model=None):
    if not os.path.exists("test.csv"):
        print("test.csv not found.")
        return
        
    df = pd.read_csv("test.csv")
    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
    dataset = LLMPreferenceDataset(df, tokenizer, max_length=Config.MAX_LENGTH, is_train=False)
    dataloader = DataLoader(dataset, batch_size=Config.BATCH_SIZE, shuffle=False)
    
    if model is None:
        model = LLMPreferenceModel(Config.MODEL_NAME)
        if os.path.exists("best_model.pt"):
            model.load_state_dict(torch.load("best_model.pt", map_location=device))
        model.to(device)
        
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Inference"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            probs = torch.softmax(outputs.logits, dim=1)
            all_probs.append(probs.cpu().numpy())
            
    all_probs = np.concatenate(all_probs, axis=0)
    
    submission = pd.DataFrame({
        'id': df['id'],
        'winner_model_a': all_probs[:, 0],
        'winner_model_b': all_probs[:, 1],
        'winner_tie': all_probs[:, 2]
    })
    
    submission.to_csv("submission.csv", index=False)
    print("Submission saved!")
    print(submission.head())

In [ ]:
# Run Pipeline
if __name__ == "__main__":
    trained_model = train()
    inference(trained_model)